In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import copy

import numpy as np
from scipy import ndimage
from scipy.spatial import cKDTree

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table, Column, vstack
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import gridspec

# Preparing for BGS Y4-6 Prediction

- Checking Anand Raichoor's `fiberassign` files
- Checking the completeness of the BGS targets

In [4]:
loc = '/Users/song/galaxy/desi/fiberassign'

sgc_cat = Table.read(os.path.join(loc, 'bright-sgc/targ-after-pass5.fits'))

In [6]:
print(sgc_cat.colnames)

['RA', 'DEC', 'TARGETID', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SUBPRIORITY', 'OBSCONDITIONS', 'PRIORITY_INIT', 'NUMOBS_INIT', 'SCND_TARGET', 'NFIBER_AVAIL', 'NTILE_AVAIL', 'NUMOBS_DONE', 'ISRADEC_SAFE', 'NUMOBS_MORE', 'NUMOBS', 'Z', 'ZWARN', 'ZTILEID', 'Z_QN', 'IS_QSO_QN', 'DELTACHI2', 'TARGET_STATE', 'TIMESTAMP', 'VERSION', 'PRIORITY']


In [11]:
# BGS in SGC
sgc_bgs = sgc_cat[sgc_cat['BGS_TARGET'] > 0]

print(len(sgc_bgs))

6320104


In [33]:
# BGS in NGC
ngc_cat = Table.read(os.path.join(loc, 'bright-ngc/targ-after-pass5.fits'))

ngc_bgs = ngc_cat[ngc_cat['BGS_TARGET'] > 0]

print(len(ngc_bgs))

14187580


In [43]:
bgs_cat = vstack([ngc_bgs, sgc_bgs])

print(len(bgs_cat))

20507684


In [44]:
bgs_cat[1]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,SCND_TARGET,NFIBER_AVAIL,NTILE_AVAIL,NUMOBS_DONE,ISRADEC_SAFE,NUMOBS_MORE,NUMOBS,Z,ZWARN,ZTILEID,Z_QN,IS_QSO_QN,DELTACHI2,TARGET_STATE,TIMESTAMP,VERSION,PRIORITY
float64,float64,int64,int64,int64,int64,float64,int32,int64,int64,int64,int64[6],int64[6],int64[6],bool,int64,int64,float64,int64,int32,float64,int16,float64,bytes30,bytes25,bytes14,int64
129.94478196513236,4.337557713578675,39627889548722586,1152921504606846976,131074,0,0.8308635630938688,516,2100,2,0,0 .. 0,0 .. 0,0 .. 0,True,0,1,-1.0,0,1,-1.0,-1,-1.0,BGS_BRIGHT|MORE_ZGOOD,2022-05-16T08:10:02+00:00,2.5.0,2


- `BGS_TARGET` bitmasks:
    - `1`: `BGS_FAINT`
    - `2`: `BGS_BRIGHT`

In [54]:
bgs_bright = bgs_cat[(bgs_cat["BGS_TARGET"] & 2) != 0]
bgs_faint = bgs_cat[(bgs_cat["BGS_TARGET"] & 1) != 0]

print(len(bgs_bright), len(bgs_faint))

12656074 7806053


In [56]:
area = 14861.0 # deg^2

In [65]:
# After Pass 1
bgs_bright_pass1 = bgs_bright['NUMOBS_DONE'][:, 0:1].sum(axis=1)
print((bgs_bright_pass1 > 0).sum() / len(bgs_bright_pass1))

# After Pass 2
bgs_bright_pass2 = bgs_bright['NUMOBS_DONE'][:, 0:2].sum(axis=1)
print((bgs_bright_pass2 > 0).sum() / len(bgs_bright_pass2))

# After Pass 3
bgs_bright_pass3 = bgs_bright['NUMOBS_DONE'][:, 0:3].sum(axis=1)
print((bgs_bright_pass3 > 0).sum() / len(bgs_bright_pass3))

# After Pass 4
bgs_bright_pass4 = bgs_bright['NUMOBS_DONE'][:, 0:4].sum(axis=1)
print((bgs_bright_pass4 > 0).sum() / len(bgs_bright_pass4))

# After Pass 5
bgs_bright_pass5 = bgs_bright['NUMOBS_DONE'][:, 0:5].sum(axis=1)
print((bgs_bright_pass5 > 0).sum() / len(bgs_bright_pass5))

# After Pass 6
bgs_bright_pass6 = bgs_bright['NUMOBS_DONE'][:, 0:6].sum(axis=1)
print((bgs_bright_pass6 > 0).sum() / len(bgs_bright_pass6))

0.2861090255951411
0.5084110601755331
0.6710901026653289
0.7864212867276219
0.8612050624862023
0.9095186232318174


In [66]:
# After Pass 1
bgs_faint_pass1 = bgs_faint['NUMOBS_DONE'][:, 0:1].sum(axis=1)
print((bgs_faint_pass1 > 0).sum() / len(bgs_faint_pass1))

# After Pass 2
bgs_faint_pass2 = bgs_faint['NUMOBS_DONE'][:, 0:2].sum(axis=1)
print((bgs_faint_pass2 > 0).sum() / len(bgs_faint_pass2))

# After Pass 3
bgs_faint_pass3 = bgs_faint['NUMOBS_DONE'][:, 0:3].sum(axis=1)
print((bgs_faint_pass3 > 0).sum() / len(bgs_faint_pass3))

# After Pass 4
bgs_faint_pass4 = bgs_faint['NUMOBS_DONE'][:, 0:4].sum(axis=1)
print((bgs_faint_pass4 > 0).sum() / len(bgs_faint_pass4))

# After Pass 5
bgs_faint_pass5 = bgs_faint['NUMOBS_DONE'][:, 0:5].sum(axis=1)
print((bgs_faint_pass5 > 0).sum() / len(bgs_faint_pass5))

# After Pass 6
bgs_faint_pass6 = bgs_faint['NUMOBS_DONE'][:, 0:6].sum(axis=1)
print((bgs_faint_pass6 > 0).sum() / len(bgs_faint_pass6))

0.14809443389636223
0.30674196037357165
0.4621377794898395
0.6003922853201227
0.7123512996901251
0.7976510023695714
